In [3]:
import numpy as np

def inner_rows(C,A,B):
    for i in range(len(A)):
        for j in range(len(A)):
            C[i,j] = A[i,j] + B[i,j]

def inner_cols(C,A,B):
    for j in range(len(A)):
        for i in range(len(A)):
            C[i,j] = A[i,j] + B[i,j]


def inner_alloc(C,A,B):
    for i in range(len(A)):
        for j in range(len(A)):
            val = [A[i,j] + B[i,j]]
            C[i,j] = val[0]


In [4]:
A = np.random.rand(100,100)
B = np.random.rand(100,100)
C = np.random.rand(100,100)


In [5]:
%timeit inner_rows(C,A,B)
%timeit inner_cols(C,A,B)
%timeit inner_alloc(C,A,B)

3.34 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.29 ms ± 67.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.8 ms ± 5.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%reload_ext memory_profiler

In [7]:
%memit inner_rows(C,A,B)
%memit inner_cols(C,A,B)
%memit inner_alloc(C,A,B)

peak memory: 106.00 MiB, increment: 0.25 MiB
peak memory: 106.00 MiB, increment: 0.00 MiB
peak memory: 106.00 MiB, increment: 0.00 MiB


## Menggunakan Decorator JIT

In [8]:
import numba
import numpy as np

In [9]:
def py_sum(x):
    hasil = 0
    for i in range(len(x)):
        hasil = hasil + x[i]
    return hasil

In [10]:
@numba.jit(nopython=True) # Decorator Numba
def numba_sum(x):
    hasil = 0
    for i in range(len(x)):
        hasil = hasil + x[i]
    return hasil

In [11]:
# generating data
x = np.random.randint(10, 100, 100_000)
x.shape

(100000,)

In [12]:
%timeit py_sum(x)

11.5 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%timeit numba_sum(x)

26.6 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Latihan

Buat fungsi `numba_sum(x, y)` untuk menghitung jarak-$L_1$ 

$$
L_1 = \sum_{i=0}^{N-1} |x_i - y_i|.
$$

Kemudian, buat perbandingan waktu komputasi antara **python original**, menggunakan **numpy.sum**, dan **numba**. Gunakan modul `from time import time` untuk menghitung waktu eksekusinya.

In [1]:
import numpy as np
import numba

In [2]:
# Generate 1 juta bilangan random (0,9)
x = np.random.randint(0, 9, 1_000_000)
y = np.random.randint(0, 9, 1_000_000)

In [3]:
def py_sum(x, y):
    hasil = 0
    for i in range(len(x)):
        hasil = hasil + abs(x[i] - y[i])
    return hasil

In [4]:
@numba.njit
def numba_sum(x, y):
    hasil = 0
    for i in range(len(x)):
        hasil = hasil + abs(x[i] - y[i])
    return hasil

In [5]:
py_sum(x, y)

2967748

In [6]:
from time import time

start = time()
py_sum(x, y)
exec_py = time() - start
print("waktu eksekusi python: {} detik".format(exec_py))

start = time()
np.sum(np.abs(x-y))
exec_np = time() - start
print("waktu eksekusi numpy: {} detik".format(exec_np))

start = time()
numba_sum(x, y)
exec_numba = time() - start
print("waktu eksekusi numba: {} detik".format(exec_numba))

waktu eksekusi python: 0.2772200107574463 detik
waktu eksekusi numpy: 0.0044481754302978516 detik
waktu eksekusi numba: 0.2458207607269287 detik


Jalankan 10 kali perhitungan di atas dan simpan hasilnya kemudian tampilkan nilai rata-rata dan standard deviasi dari perhitungan tersebut.

In [7]:
from time import time

hasil_py_sum = np.zeros(shape=10)
hasil_np_sum = np.zeros(shape=10)
hasil_numba_sum = np.zeros(shape=10)

for i in range(10):
    start = time()
    py_sum(x, y)
    exec_py = time() - start
    hasil_py_sum[i] = exec_py

    start = time()
    np.sum(np.abs(x-y))
    exec_np = time() - start
    hasil_np_sum[i] = exec_np

    start = time()
    numba_sum(x, y)
    exec_numba = time() - start
    hasil_numba_sum[i] = exec_numba


In [8]:
print(hasil_py_sum)
print(hasil_np_sum)
print(hasil_numba_sum)

[0.27235818 0.26828504 0.26779318 0.26770687 0.26844525 0.26766396
 0.26736617 0.26739883 0.2670362  0.26742101]
[0.00675392 0.00316119 0.00300789 0.00273991 0.00275517 0.00270009
 0.0031321  0.00319886 0.00269699 0.00277996]
[0.00038505 0.00033212 0.00033307 0.00033522 0.00033212 0.00035882
 0.00033498 0.00033402 0.00033379 0.00033498]


In [12]:
speedup_python = hasil_py_sum / hasil_numba_sum
speedup_np = hasil_np_sum / hasil_numba_sum

mean_py = np.mean(speedup_python)
std_py = np.std(speedup_python)
mean_np = np.mean(speedup_np)
std_np = np.std(speedup_np)

print("Speedup (Python, Numba): mean = {}, std.dev = {}".format(mean_py, std_py))
print("Speedup (NumPy, Numba): mean = {}, std.dev = {}".format(mean_np, std_np))

Speedup (Python, Numba): mean = 786.9047638004301, std.dev = 31.537170359724144
Speedup (NumPy, Numba): mean = 9.538970800399657, std.dev = 2.74688552921028
